# blog 크롤링

## 크롤링 조건 설정

In [54]:
# 크롤링할 페이지 설정
number_page = 4
# 날짜 설정 : "설정 날짜" 포함, 이후 날짜 데이터 출력됨.
start_date = '2022-01-01'

In [74]:
from bs4 import BeautifulSoup as bs
import time
import random
from selenium import webdriver
import pandas as pd
from datetime import datetime

## 크롤링할 페이지 접속

In [56]:
# 페이지 접속
driver = webdriver.Chrome('chromedriver')
url = 'https://www.gyeonggido-korea.com/'
# webdriver 로드
driver.get(url)
time.sleep(3)
driver.implicitly_wait(2)

# 창 최대화
driver.maximize_window()
time.sleep(1)

In [57]:
# 페이지가 스프에 담김 확인
soup = bs(driver.page_source, 'lxml')
print("타이틀 : ", soup.title.text)

타이틀 :  Gyeonggido-Korea (경기도)


In [58]:
# about 페이지로 이동
view_button = driver.find_element_by_xpath('//*[@id="Label1"]/div[1]/div[1]/a/span')
view_button.click()
time.sleep(1)

<ipython-input-58-682365819f32>:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  view_button = driver.find_element_by_xpath('//*[@id="Label1"]/div[1]/div[1]/a/span')


## 페이지 이동 버튼 링크 수집
* 링크 수집해서 이동하는 방법 : 주소창에 링크가 들어가지만 페이지 이동은 하지 않음
* 이동 버튼 xpath를 가져와 클릭해서 페이지 이동 구현

In [60]:
# 페이지 이동 버튼 xpath 수집
page_link_xpaths = driver.find_elements_by_xpath('//*[@id="HTML4"]/div[3]/a')
print("페이지 이동버튼 개수 : ", len(page_link_xpaths))

페이지 이동버튼 개수 :  10


<ipython-input-60-0a50c60668da>:2: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  page_link_xpaths = driver.find_elements_by_xpath('//*[@id="HTML4"]/div[3]/a')


## 각 페이지에서 게시글 정보 크롤링

In [62]:
# 각 페이지에서 게시글 크롤링
page_title_list = []
page_date_list = []
post_link_list = []

for i in range(len(page_link_xpaths[:number_page])):
    page_button = driver.find_element_by_xpath(f'//*[@id="HTML4"]/div[3]/a[{i+1}]')
    page_button.click()
    driver.implicitly_wait(3)

    time.sleep( random.uniform(1, 2) ) # 크롤링 차단 방지
    page_soup = bs(driver.page_source, 'lxml')

    # 게시글 제목 수집
    titles = page_soup.find_all('h3', 'item-title')
    
    for title in titles:
        temp = title.find('a').text
        page_title_list.append(temp)

    # 게시글 날짜 수집
    dates = page_soup.find_all('div', 'meta-items')

    for date in dates:
        temp = date.find('a', 'meta-item meta-item-date').find('span').text
        page_date_list.append(temp)

    # 게시글 링크 수집
    for link in titles:
        temp = link.find('a').attrs['href']
        post_link_list.append(temp)

  0%|          | 0/4 [00:00<?, ?it/s]<ipython-input-62-293e1ca15f4f>:7: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  page_button = driver.find_element_by_xpath(f'//*[@id="HTML4"]/div[3]/a[{i+1}]')
100%|██████████| 4/4 [00:06<00:00,  1.55s/it]


In [63]:
# chromedriver 종료
driver.quit()

## 수집한 데이터 확인

In [64]:
print("수집한 제목 수 : ", len(page_title_list))
print("수집한 날짜 수 : ", len(page_date_list))
print("수집한 링크 수 : ", len(post_link_list))

수집한 제목 수 :  28
수집한 날짜 수 :  28
수집한 링크 수 :  28


## 날짜 형식 변환

In [76]:
mod_page_date_list = []
for raw_date in page_date_list:
    temp = raw_date.replace(',', '')
    temp_list = temp.split(' ')
    year = temp_list[2]
    day = temp_list[1]
    dic = {'Jan':'01', 'Feb':'02', 'Mar':'03', 'Apr':'04', 'May':'05', 'Jun':'06', 'Jul':'07', 'Aug':'08', 'Sep':'09', 'Oct':'10', 'Nov':'11', 'Dec':'12'}
    month = dic[temp_list[0]]
    date = f'{year}-{month}-{day}'
    mod_page_date_list.append(date)

print("수정된 날짜 개수 : ", len(mod_page_date_list))    

수정된 날짜 개수 :  28


## 조회수, 좋아요 수, 공유수, 댓글 수 기본 리스트 생성

In [79]:
number_view_list = []
number_like_list = []
number_share_list = []
number_comment_list = []

for i in range(len(mod_page_date_list)):
    number_view_list.append(0)
for i in range(len(mod_page_date_list)):
    number_like_list.append(0)
for i in range(len(mod_page_date_list)):
    number_share_list.append(0)
for i in range(len(mod_page_date_list)):
    number_comment_list.append(0)

print("수정된 날짜 개수 : ", len(mod_page_date_list))
print("조회수 리스트 원소 개수 : ", len(mod_page_date_list))
print("좋아요 리스트 원소 개수 : ", len(mod_page_date_list))
print("공유 리스트 원소 개수 : ", len(mod_page_date_list))
print("댓글 리스트 원소 개수 : ", len(mod_page_date_list))

수정된 날짜 개수 :  28
조회수 리스트 원소 개수 :  28
좋아요 리스트 원소 개수 :  28
공유 리스트 원소 개수 :  28
댓글 리스트 원소 개수 :  28


## 데이터 셋 생성

In [80]:
# 데이터 프레임 생성
df = {'날짜':mod_page_date_list, '제목':page_title_list, '조회수':number_view_list, '좋아요_수':number_like_list, '공유수':number_share_list, '댓글수':number_comment_list, '링크':post_link_list}
data = pd.DataFrame(df)
data

,날짜,제목,조회수,좋아요_수,공유수,댓글수,링크
0,2022-03-23,Gyeonggi Builds and Operates Korea’s First Fir...,0,0,0,0,https://www.gyeonggido-korea.com/2022/03/gyeon...
1,2022-03-21,"A Hot Place in Gyeonggi-do, Flower Mart, Launc...",0,0,0,0,https://www.gyeonggido-korea.com/2022/03/a-hot...
2,2022-03-19,Gyeonggi Provides QR Codes and Foreign Languag...,0,0,0,0,https://www.gyeonggido-korea.com/2022/03/gyeon...
3,2022-03-17,Gyeonggi Provides Multicultural Newspaper Subs...,0,0,0,0,https://www.gyeonggido-korea.com/2022/03/gyeon...
4,2022-03-16,2022 Gyeonggi Foreign Social Media Reporters R...,0,0,0,0,https://www.gyeonggido-korea.com/2022/03/2022-...
5,2022-03-15,Gyeonggi Releases 1.8 Million Data Autonomous ...,0,0,0,0,https://www.gyeonggido-korea.com/2022/03/gyeon...
6,2022-03-12,Gyeonggi Province’s Namyangju City launches mu...,0,0,0,0,https://www.gyeonggido-korea.com/2022/03/gyeon...
7,2022-03-10,Gyeonggi Province improves foreign laborer she...,0,0,0,0,https://www.gyeonggido-korea.com/2022/03/gyeon...
8,2022-03-08,Gyeonggi Province's Gwangju City adopts VOICEY...,0,0,0,0,https://www.gyeonggido-korea.com/2022/03/gyeon...
9,2022-03-04,Gyeonggi Province’s Ansan City provides interp...,0,0,0,0,https://www.gyeonggido-korea.com/2022/03/gyeon...


## 데이터 셋. 날짜로 슬라이싱

In [81]:
# 데이터 셋 날짜로 슬라이싱
slicing_data = data[data['날짜'] >= start_date]
slicing_data

,날짜,제목,조회수,좋아요_수,공유수,댓글수,링크
0,2022-03-23,Gyeonggi Builds and Operates Korea’s First Fir...,0,0,0,0,https://www.gyeonggido-korea.com/2022/03/gyeon...
1,2022-03-21,"A Hot Place in Gyeonggi-do, Flower Mart, Launc...",0,0,0,0,https://www.gyeonggido-korea.com/2022/03/a-hot...
2,2022-03-19,Gyeonggi Provides QR Codes and Foreign Languag...,0,0,0,0,https://www.gyeonggido-korea.com/2022/03/gyeon...
3,2022-03-17,Gyeonggi Provides Multicultural Newspaper Subs...,0,0,0,0,https://www.gyeonggido-korea.com/2022/03/gyeon...
4,2022-03-16,2022 Gyeonggi Foreign Social Media Reporters R...,0,0,0,0,https://www.gyeonggido-korea.com/2022/03/2022-...
5,2022-03-15,Gyeonggi Releases 1.8 Million Data Autonomous ...,0,0,0,0,https://www.gyeonggido-korea.com/2022/03/gyeon...
6,2022-03-12,Gyeonggi Province’s Namyangju City launches mu...,0,0,0,0,https://www.gyeonggido-korea.com/2022/03/gyeon...
7,2022-03-10,Gyeonggi Province improves foreign laborer she...,0,0,0,0,https://www.gyeonggido-korea.com/2022/03/gyeon...
8,2022-03-08,Gyeonggi Province's Gwangju City adopts VOICEY...,0,0,0,0,https://www.gyeonggido-korea.com/2022/03/gyeon...
9,2022-03-04,Gyeonggi Province’s Ansan City provides interp...,0,0,0,0,https://www.gyeonggido-korea.com/2022/03/gyeon...


# 데이터 셋. 인덱스 내림차순 정렬

In [82]:
# 인덱스 내림차순 정렬
final_data = slicing_data.sort_index(ascending=False)
final_data

,날짜,제목,조회수,좋아요_수,공유수,댓글수,링크
23,2022-01-20,Experience the electronics industry with the S...,0,0,0,0,https://www.gyeonggido-korea.com/2022/01/exper...
22,2022-01-24,Gyeonggi Province leads vitalization of health...,0,0,0,0,https://www.gyeonggido-korea.com/2022/01/gyeon...
21,2022-01-26,Suwon in the world: Places to visit for living...,0,0,0,0,https://www.gyeonggido-korea.com/2022/01/suwon...
20,2022-02-04,Gyeonggi Province supports foreign children’s ...,0,0,0,0,https://www.gyeonggido-korea.com/2022/02/gyeon...
19,2022-02-06,Gyeonggi Province implements emergency welfare...,0,0,0,0,https://www.gyeonggido-korea.com/2022/02/gyeon...
18,2022-02-08,"In Pyeongtaek City of Gyeonggi Province, you c...",0,0,0,0,https://www.gyeonggido-korea.com/2022/02/in-py...
17,2022-02-10,Paju City of Gyeonggi Province - Providing hap...,0,0,0,0,https://www.gyeonggido-korea.com/2022/02/paju-...
16,2022-02-11,All foreign residents of Gyeonggi Province Ple...,0,0,0,0,https://www.gyeonggido-korea.com/2022/02/all-f...
15,2022-02-15,Ansan City of Gyeonggi Province will expand ch...,0,0,0,0,https://www.gyeonggido-korea.com/2022/02/ansan...
14,2022-02-18,Gimpo City of Gyeonggi Province adopts expat s...,0,0,0,0,https://www.gyeonggido-korea.com/2022/02/gimpo...


## 엑셀 파일로 출력

In [83]:
# 파일로 출력
file_name = str(datetime.today().strftime("%Y%m%d-%H%M%S"))
file_name = 'blog_' + file_name + '.xlsx'
path = './crawling_data/blog/'
final_data.to_excel(path+file_name, index=False, encoding='utf-8')

print(f"{file_name} 파일 생성 완료")

blog_20220324-120013.xlsx 파일 생성 완료


# 코드 마지막